# Shyam Parikh
## I will be analyzing Cluster 0 for this project

In [1]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv('../data/train_data_transformed.csv')
df = df[df['Cluster'] == 0]
df

,Cash Flow to Equity,Retained Earnings to Total Assets,Revenue per person,Current Liability to Current Assets,Equity to Liability,Cash/Total Assets,Total expense/Assets,Liability-Assets Flag,Total debt/Total net worth,Operating profit per person,...,Current Ratio,Tax rate (A),Fixed Assets to Assets,Fixed Assets Turnover Frequency,Operating Expense Rate,Cash Turnover Rate,Cash/Current Liability,Net Income Flag,Cluster,Bankrupt?
1,0.216729,-0.034301,0.016897,0.019162,0.021986,0.051009,0.014031,0.0,-7.698724,0.222590,...,0.008968,0.140670,0.185825,0.000286,0.000102,47.794965,0.003365,1,0,0
7,0.215399,-0.041921,0.005280,0.027536,0.022067,0.017753,0.015424,0.0,-7.716754,0.220964,...,0.005558,0.000000,0.239122,4.128375,0.000178,46.147403,0.001144,1,0,0
8,0.215869,-0.039591,0.009694,0.022347,0.026473,0.032683,0.009806,0.0,-8.676308,0.222620,...,0.007488,0.104333,0.238954,4.115734,0.000131,49.288790,0.002932,1,0,0
10,0.216380,-0.039687,0.008022,0.022317,0.025973,0.028869,0.010690,0.0,-8.568886,0.220527,...,0.007500,0.000000,0.251501,4.127830,0.000141,48.305930,0.001913,1,0,0
11,0.215860,-0.042882,0.027296,0.018954,0.022168,0.009289,0.019270,0.0,-7.739710,0.219033,...,0.009075,0.000000,0.073640,0.000717,0.000126,41.385288,0.000724,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5794,0.215657,-0.044093,0.004741,0.034058,0.016710,0.011484,0.014059,0.0,-6.359663,0.220388,...,0.003639,0.000000,0.156496,0.000116,0.000228,45.874156,0.000514,1,0,0
5795,0.214344,-0.038055,0.023649,0.008283,0.035300,0.064855,0.013428,0.0,-10.859020,0.223973,...,0.017268,0.061093,0.059463,0.000876,0.000135,50.765054,0.010702,1,0,0
5796,0.216097,-0.033410,0.006366,0.028012,0.029011,0.036732,0.017590,0.0,-9.227730,0.222447,...,0.005404,0.123248,0.224897,0.000102,0.000301,49.514987,0.003687,1,0,0
5798,0.215324,-0.041749,0.009597,0.024928,0.023986,0.011364,0.017159,0.0,-8.140899,0.221075,...,0.006467,0.199058,0.286477,0.000129,0.000104,41.806174,0.000781,1,0,0
